In [59]:
import sys

%reload_ext autoreload
%autoreload 2

sys.path.insert(0, '../..')

In [60]:
from typing import Union, Tuple

import os
import pandas as pd
import json
import numpy as np
from sklearn.preprocessing import MinMaxScaler

import faiss
import tensorflow_hub as hub

import torch
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample
)

from moviepy.editor import VideoFileClip

from img2vec_pytorch import Img2Vec
from PIL import Image

import warnings
warnings.filterwarnings('ignore')

from utils import *
from base import Recommendation

In [61]:
root_path = '../../data/artifacts'

In [4]:
class Metadata(Recommendation):
    """
    Using text, images and videos to identify interest sub-categories
    """
    
    device = "cpu" # set to cuda to load on GPU

    @property
    def processed_file_names(self) -> Union[str, List[str], Tuple]:
        """The name of the files in the processed_path folder that
        must be present in order to skip training."""
        return ['multimodal_adventures.p',
                'multimodal_img_faiss.index',
                'multimodal_text_faiss.index',
                'kinetics_id_to_classname.p']


    def init_processed_paths(self):
        """
        initiate the file paths for processed data
        """
        self.processed_path_adventures = os.path.join(self.processed_dir, 'multimodal_adventures.p')
        self.processed_path_img_faiss_index = os.path.join(self.processed_dir, 'multimodal_img_faiss.index')
        self.processed_path_text_faiss_index = os.path.join(self.processed_dir, 'multimodal_text_faiss.index')
        self.processed_path_model_use_v5 = os.path.join(self.processed_dir, 'models/pretrained/use_v5')
        self.processed_path_model_torch_model_zoo = os.path.join(self.processed_dir, 'models/pretrained/torch_model_zoo')
        self.processed_path_kinetics_id_to_classname = os.path.join(self.processed_dir, 'kinetics_id_to_classname.p')

        os.environ['TORCH_HOME'] = self.processed_path_model_torch_model_zoo


    def process(self):
        """
        load the raw data, process it and save into processed data folder
        """
        # load the adventure interest sub-category table
        adventures = pd.read_csv(self.raw_path_adventures)

        # load the text vectorisation model
        embed = hub.KerasLayer(self.processed_path_model_use_v5)
        
        # process and save the adventures table
        adventures = adventures[['id', 'name', 'icon', 'parent_id']]
        adventures['save_path'] = adventures['icon'].apply(get_img_path, basepath=self.raw_path_images_adventures)
        save_pickle(adventures, self.processed_path_adventures)

        # convert text to vectors and save
        embeddings = embed(adventures['name'].tolist()).numpy()
        index_flat_text = IndexFlatL2(512, adventures, embeddings)
        index_flat_text.build()
        faiss.write_index(index_flat_text.index, self.processed_path_text_faiss_index)

        # convert image to vectors and save
        BUFFERSIZE = 100
        img_paths = adventures.save_path.tolist()
        img2vec = Img2Vec(cuda=False)
        img_vecs = None
        for i in range(0, len(img_paths), BUFFERSIZE):
            max_range = (i+BUFFERSIZE) if (i+BUFFERSIZE) <= len(img_paths) else len(img_paths)
            _img_paths = img_paths[i:i+BUFFERSIZE]
            _vectors = img2vec.get_vec([Image.open(ipath).convert('RGB') for ipath in _img_paths])
            if img_vecs is None:
                img_vecs = _vectors
            else:
                img_vecs = np.vstack((img_vecs, _vectors))
                
        index_flat_img = IndexFlatL2(512, adventures, img_vecs)
        index_flat_img.build()
        faiss.write_index(index_flat_img.index, self.processed_path_img_faiss_index)
        
        with open(self.raw_path_kinetics_classnames , "r") as f:
            kinetics_classnames = json.load(f)

        # Create an id to label name mapping
        kinetics_id_to_classname = {}
        for k, v in kinetics_classnames.items():
            kinetics_id_to_classname[v] = str(k).replace('"', "")

        kinetics_id_to_classname = pd.DataFrame(kinetics_id_to_classname.items(), columns=['id','label']).sort_values(by='id')
        kinetics_id_to_classname.set_index('id', inplace=True)
        kinetics_id_to_classname.to_pickle(self.processed_path_kinetics_id_to_classname)


    def load(self):
        """
        load the processed data from processed data folder into memory
        """
        self.adventures = load_pickle(self.processed_path_adventures)
        self.embed = hub.KerasLayer(self.processed_path_model_use_v5)
        img2vec = Img2Vec(cuda=False)
        self.img_embed = lambda x: img2vec.get_vec(x)
        text_index = faiss.read_index(self.processed_path_text_faiss_index)
        self.index_flat_text = IndexFlatL2(512, self.adventures, index=text_index)
        self.index_flat_text.build()
        img_index = faiss.read_index(self.processed_path_img_faiss_index)
        self.index_flat_img = IndexFlatL2(512, self.adventures, index=img_index)
        self.index_flat_img.build()
        
        self.kinetics_id_to_classname = pd.read_pickle(self.processed_path_kinetics_id_to_classname)

        device = self.device
        model_name = "x3d_xs"
        mean = [0.45, 0.45, 0.45]
        std = [0.225, 0.225, 0.225]
        frames_per_second = 30
        model_transform_params  = {
            "x3d_xs": {
                "side_size": 182,
                "crop_size": 182,
                "num_frames": 4,
                "sampling_rate": 12,
            }
        }
    
        self.model = torch.hub.load("facebookresearch/pytorchvideo:main",
                                    model=model_name,
                                    pretrained=True)
        # set to eval mode and move to desired device
        self.model = self.model.to(device)
        self.model = self.model.eval()

        transform_params = model_transform_params[model_name]

        self.transform =  ApplyTransformToKey(
            key="video",
            transform=Compose(
                [
                    UniformTemporalSubsample(transform_params["num_frames"]),
                    Lambda(lambda x: x/255.0),
                    NormalizeVideo(mean, std),
                    ShortSideScale(size=transform_params["side_size"]),
                    CenterCropVideo(
                        crop_size=(transform_params["crop_size"], transform_params["crop_size"])
                    )
                ]
            ),
        )

        # duration of the input clip is specific to the model
        self.clip_duration = (transform_params["num_frames"] * transform_params["sampling_rate"])/frames_per_second


    def classify_video(self,
                       video_path,
                       topk = 5,
                       headstart = 0,
                       limit = 60,
                       verbose = True,
                       ):
        
        pred_class_names_all = []

        
        clip_length = VideoFileClip(video_path).duration # in seconds
        clip_length = clip_length - headstart
        clip_length = clip_length if clip_length < limit else limit

        segments = int(clip_length // self.clip_duration)

        for i in range(0, segments):

            start_sec = headstart + i * self.clip_duration

            # Select the duration of the clip to load by specifying the start and end duration
            # The start_sec should correspond to where the action occurs in the video
            end_sec = start_sec + self.clip_duration

            if verbose:
                print('Analysing {:.2f}s-{:.2f}s clip segment | Segment {}/{}'\
                    .format(start_sec, end_sec, i+1, segments))

            # Initialize an EncodedVideo helper class
            video = EncodedVideo.from_path(video_path)

            # Load the desired clip
            video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)

            # Apply a transform to normalize the video input
            video_data = self.transform(video_data)

            # Move the inputs to the desired device
            inputs = video_data["video"]
            inputs = inputs.to(self.device)[None, ...] # for X3D model

            # Pass the input clip through the model 
            preds = self.model(inputs)

            # Get the predicted classes 
            post_act = torch.nn.Softmax(dim=1)
            preds = post_act(preds)
            pred_classes = preds.topk(k=topk).indices

            # Map the predicted classes to the label names
            pred_class_names = self.kinetics_id_to_classname.loc[[int(i) for i in pred_classes[0]], 'label'].values.tolist()
            pred_class_names_all.extend(pred_class_names)

        return pred_class_names_all
    

    def recommend(self,
                    title : str = None,
                    description : str = None,
                    image_path : str = None,
                    video_path : str = None,
                    top_k : int = 10,
                    threshold : int = 50,
                    weights : dict = None,
                    return_df : bool = False,
                    headstart = 10,
                    limit = 30,
                    verbose = False):
        """
        For the given title, description, image path,
        and video path, 
        identify and return the interest sub-categories
        """
        # get top-k interest sub-categories for the title
        title_candidates = pd.DataFrame(columns=['name','distance'])
        if title is not None:
            title_candidates = pd.DataFrame([{'name':x[1]['name'], 'distance':x[0]} for x in \
                                                get_ann_top_items(self.embed, self.index_flat_text, 
                                                                title, threshold)])

        # get top-k interest sub-categories for the description
        description_candidates = pd.DataFrame(columns=['name','distance'])
        if description is not None:
            description_candidates = pd.DataFrame([{'name':x[1]['name'], 'distance':x[0]} for x in \
                                                    get_ann_top_items(self.embed, self.index_flat_text, 
                                                                        description, threshold)])

        # get top-k interest sub-categories for the image
        image_candidates = pd.DataFrame(columns=['name','distance_image'])
        if image_path is not None:
            image_path = download_images(image_path, basepath=os.path.join(self.root_path, 'raw/images'))
            image_candidates = pd.DataFrame([{'name':x[1]['name'], 'distance_image':x[0]} for x in \
                                                get_ann_top_items_img(self.img_embed, self.index_flat_img, 
                                                                    image_path, threshold)])

        # get top-k interest sub-categories for the videos
        video_candidates = pd.DataFrame(columns=['name','distance_video'])
        video_weights = {'video': 1}
        
        if video_path is not None:
            video_path = download_videos(video_path, basepath=os.path.join(self.root_path, 'raw/videos'))
            # get list of labels in video
            labels = self.classify_video(video_path,
                                        headstart=headstart,
                                        limit=limit,
                                        verbose=verbose)
            
            # convert list to countdict
            labels = {x:labels.count(x) for x in labels}
            
            # convert dict to df
            labels = pd.DataFrame(labels.items(), columns=['label','count'])

            # select top-10 labels
            n_labels = 10
            labels = labels.sort_values(by='count', ascending=False).head(n_labels)

            video_weights = labels.set_index('label').to_dict()['count']
            col_names = labels['label'].tolist()
            
            video_candidates = pd.DataFrame(columns=['name','distance'])

            for index, row in labels.iterrows():
                _df = pd.DataFrame([{'name':x[1]['name'], 'distance':x[0]} for x in \
                    get_ann_top_items(self.embed, self.index_flat_text,
                                        row.label, threshold)])
                video_candidates = video_candidates.merge(_df, on='name', how='outer')

            video_candidates.set_index('name', inplace=True)
            video_candidates = video_candidates.iloc[: , 1:]
            video_candidates.columns = col_names

        # merge
        candidates = title_candidates.merge(description_candidates, on='name', how='outer', 
                                            suffixes=('_title','_description'))
        candidates = candidates.merge(image_candidates, on='name', how='outer')
        candidates = candidates.merge(video_candidates, on='name', how='outer')
        candidates.set_index('name', inplace=True)
        
        if weights is None:
            weights = {'title': 1, 'description': 1, 'image': 1}
        # add video candidate weights
        weights.update(video_weights)
        
        col_names = candidates.columns.tolist()

        # combine
        min_max_scaler = MinMaxScaler()
        x_scaled = min_max_scaler.fit_transform(candidates.values)
        candidates = pd.DataFrame(x_scaled, index=candidates.index)

        candidates.columns = col_names
        candidates = candidates.rename(columns={'distance_title':'title',
                                                    'distance_description':'description',
                                                    'distance_image':'image',
                                                    'distance_video':'video'})
        candidates.fillna(1, inplace=True)
        candidates = candidates.astype('float')
        candidates.replace(0, 1e-2, inplace=True)
        candidates = 1/np.sqrt(candidates)
        candidates['final_score'] = np.array([(candidates[x]*weights[x]).values for x in \
                                                candidates.columns]).sum(axis=0)
        candidates.sort_values(by='final_score', ascending=False, inplace=True)
        
        candidates = candidates[~candidates.index.duplicated(keep='first')]

        # selecting top-k
        candidates = candidates.head(top_k)

        if return_df:
            return candidates.to_json()

        candidates = candidates.index.tolist()

        return candidates

In [ ]:
m = Metadata(root_path=root_path)

In [64]:
def recommend(self,
                title : str = None,
                description : str = None,
                image_path : str = None,
                video_path : str = None,
                top_k : int = 10,
                threshold : int = 50,
                weights : dict = None,
                return_df : bool = False,
                headstart = 10,
                limit = 30,
                verbose = False):
    """
    For the given title, description, image path,
    and video path, 
    identify and return the interest sub-categories
    """
    # get top-k interest sub-categories for the title
    title_candidates = pd.DataFrame(columns=['name','distance'])
    if title is not None:
        title_candidates = pd.DataFrame([{'name':x[1]['name'], 'distance':x[0]} for x in \
                                            get_ann_top_items(self.embed, self.index_flat_text, 
                                                            title, threshold)])

    # get top-k interest sub-categories for the description
    description_candidates = pd.DataFrame(columns=['name','distance'])
    if description is not None:
        description_candidates = pd.DataFrame([{'name':x[1]['name'], 'distance':x[0]} for x in \
                                                get_ann_top_items(self.embed, self.index_flat_text, 
                                                                    description, threshold)])

    # get top-k interest sub-categories for the image
    image_candidates = pd.DataFrame(columns=['name','distance_image'])
    if image_path is not None:
        image_path = download_images(image_path, basepath=os.path.join(self.root_path, 'raw/images/adventures'))
        image_candidates = pd.DataFrame([{'name':x[1]['name'], 'distance_image':x[0]} for x in \
                                            get_ann_top_items_img(self.img_embed, self.index_flat_img, 
                                                                image_path, threshold)])

    # get top-k interest sub-categories for the videos
    video_candidates = pd.DataFrame(columns=['name','distance_video'])
    video_weights = {'video': 1}
    
    if video_path is not None:
        video_path = download_videos(video_path, basepath=os.path.join(self.root_path, 'raw/videos/adventures'))
        print(video_path)
        # get list of labels in video
        labels = self.classify_video(video_path,
                                    headstart=headstart,
                                    limit=limit,
                                    verbose=verbose)
        
        # convert list to countdict
        labels = {x:labels.count(x) for x in labels}
        
        # convert dict to df
        labels = pd.DataFrame(labels.items(), columns=['label','count'])

        # select top-10 labels
        n_labels = 10
        labels = labels.sort_values(by='count', ascending=False).head(n_labels)

        video_weights = labels.set_index('label').to_dict()['count']
        col_names = labels['label'].tolist()
        
        video_candidates = pd.DataFrame(columns=['name','distance'])

        for index, row in labels.iterrows():
            _df = pd.DataFrame([{'name':x[1]['name'], 'distance':x[0]} for x in \
                get_ann_top_items(self.embed, self.index_flat_text,
                                    row.label, threshold)])
            video_candidates = video_candidates.merge(_df, on='name', how='outer')

        video_candidates.set_index('name', inplace=True)
        video_candidates = video_candidates.iloc[: , 1:]
        video_candidates.columns = col_names

    # merge
    candidates = title_candidates.merge(description_candidates, on='name', how='outer', 
                                        suffixes=('_title','_description'))
    candidates = candidates.merge(image_candidates, on='name', how='outer')
    candidates = candidates.merge(video_candidates, on='name', how='outer')
    candidates.set_index('name', inplace=True)
    
    if weights is None:
        weights = {'title': 1, 'description': 1, 'image': 1}
    # add video candidate weights
    weights.update(video_weights)
    
    col_names = candidates.columns.tolist()

    # combine
    min_max_scaler = MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(candidates.values)
    candidates = pd.DataFrame(x_scaled, index=candidates.index)

    candidates.columns = col_names
    candidates = candidates.rename(columns={'distance_title':'title',
                                                'distance_description':'description',
                                                'distance_image':'image',
                                                'distance_video':'video'})
    candidates.fillna(1, inplace=True)
    candidates = candidates.astype('float')
    candidates.replace(0, 1e-2, inplace=True)
    candidates = 1/np.sqrt(candidates)
    candidates['final_score'] = np.array([(candidates[x]*weights[x]).values for x in \
                                            candidates.columns]).sum(axis=0)
    candidates.sort_values(by='final_score', ascending=False, inplace=True)
    
    candidates = candidates[~candidates.index.duplicated(keep='first')]

    # selecting top-k
    candidates = candidates.head(top_k)

    if return_df:
        return candidates.to_json()

    candidates = candidates.index.tolist()

    return candidates

In [46]:
def get_recommendations():
    x = recommend(m,
        title = title,
        description = description,
        image_path = image_path,
        video_path = video_path,
        top_k = 10,
        threshold = 10,
        weights = {'title': 1, 'description': 1, 'image': 1},
        return_df = True,
        headstart = 10,
        limit = 5,
        verbose = True)

    display(pd.read_json(x))

In [47]:
title = 'rock climbing'
description = None
image_path = None
video_path = None

get_recommendations()

,title,description,image,video,final_score
Climbing Tours,10.000000,1,1,1,13.000000
Snowboard,1.116387,1,1,1,4.116387
Bike Tour,1.113904,1,1,1,4.113904
Kayaking,1.108532,1,1,1,4.108532
Parasailing and Paragliding,1.093432,1,1,1,4.093432
Swimming,1.051614,1,1,1,4.051614
Orienteering,1.048739,1,1,1,4.048739
Running Tours,1.046846,1,1,1,4.046846
Dance,1.032326,1,1,1,4.032326
Hill Walk,1.000000,1,1,1,4.000000


In [48]:
title = 'dance'
description = None
image_path = None
video_path = None

get_recommendations()

,title,description,image,video,final_score
Dance,10.000000,1,1,1,13.000000
Dance Classes,1.295236,1,1,1,4.295236
Party,1.180563,1,1,1,4.180563
Aerobics,1.177295,1,1,1,4.177295
Dress,1.131138,1,1,1,4.131138
Swimming,1.074876,1,1,1,4.074876
Dance clubs and discos,1.036151,1,1,1,4.036151
Bike,1.021352,1,1,1,4.021352
Cardio,1.007318,1,1,1,4.007318
Alcohol,1.000000,1,1,1,4.000000


In [49]:
title = 'fine-dine restaurant'
description = None
image_path = None
video_path = None

get_recommendations()

,title,description,image,video,final_score
Restaurant,10.000000,1,1,1,13.000000
Dinning bar,1.815856,1,1,1,4.815856
Bistro,1.339827,1,1,1,4.339827
Australian/Oceanian Cuisine,1.283280,1,1,1,4.283280
Antartican Cuisine,1.282326,1,1,1,4.282326
Cafe,1.103496,1,1,1,4.103496
Asian Cuisine,1.058069,1,1,1,4.058069
African Cuisine,1.006490,1,1,1,4.006490
Dinner,1.006445,1,1,1,4.006445
Wine bar,1.000000,1,1,1,4.000000


In [50]:
title = 'fine-dine restaurant'
description = 'enjoy fine-dine experience in the luxury Supremo hotel in Dubai'
image_path = None
video_path = None

get_recommendations()

,title,description,image,video,final_score
Restaurant,10.000000,1.000000,1,1,13.000000
Luxury,1.000000,10.000000,1,1,13.000000
Sultan Luxury Gold Hammam,1.000000,3.398108,1,1,6.398108
Luxury Safari,1.000000,2.062959,1,1,5.062959
Dinning bar,1.815856,1.000000,1,1,4.815856
Luxury SUV,1.000000,1.694187,1,1,4.694187
Luxury Morrocan Hammam,1.000000,1.599900,1,1,4.599900
Bistro,1.339827,1.000000,1,1,4.339827
A320 Prestige,1.000000,1.330041,1,1,4.330041
Australian/Oceanian Cuisine,1.283280,1.000000,1,1,4.283280


In [51]:
title = 'ice mountains'
description = '3-day trip to rocky snow mountains'
image_path = 'https://i.pinimg.com/originals/16/4d/fd/164dfd5093b35a17dc2519cad2954793.jpg'
video_path = None

get_recommendations()

,title,description,image,video,final_score
Ski and Snow Tours,1.612765,10.000000,10.000000,1,22.612765
Frozen Foods,10.000000,1.000000,1.000000,1,13.000000
Antartic Peninsula,1.571243,1.000000,1.000000,1,4.571243
Hiking and Camping Tours,1.000000,1.463563,1.000000,1,4.463563
Pair of skis,1.144087,1.078340,1.084006,1,4.306434
"4WD, ATV and Off-road Tours",1.000000,1.257143,1.000000,1,4.257143
Beaches,1.232973,1.000000,1.000000,1,4.232973
Indochinese,1.000000,1.000000,1.210810,1,4.210810
Climbing Tours,1.000000,1.194450,1.000000,1,4.194450
Soft Drinks,1.175346,1.000000,1.000000,1,4.175346


In [52]:
title = 'beach trip'
description = '7-day trip to indonesian beaches'
image_path = 'https://marylineg1.sg-host.com/blog/wp-content/uploads/2018/09/PINK-BEACH2.jpg'
video_path = None

get_recommendations()

,title,description,image,video,final_score
Beaches,10.000000,1.218466,10.000000,1,22.218466
Indonesian,1.000000,10.000000,1.000000,1,13.000000
Garuda Indonesia,1.000000,4.620205,1.000000,1,7.620205
Day Trips,1.597474,1.000000,1.000000,1,4.597474
Malaysian,1.000000,1.440189,1.000000,1,4.440189
Swimming,1.350673,1.000000,1.000000,1,4.350673
Bali Hot Stone Massage,1.000000,1.336275,1.000000,1,4.336275
Traditional Bali Massage,1.000000,1.322385,1.000000,1,4.322385
Full Upper Body,1.000000,1.000000,1.312164,1,4.312164
Bath Time,1.000000,1.000000,1.269031,1,4.269031


In [66]:
title = None
description = None
image_path = 'https://marylineg1.sg-host.com/blog/wp-content/uploads/2018/09/PINK-BEACH2.jpg'
video_path = None

get_recommendations()

,title,description,image,video,final_score
Beaches,1,1,10.000000,1,13.000000
Full Upper Body,1,1,1.312164,1,4.312164
Bath Time,1,1,1.269031,1,4.269031
Large Rucksack,1,1,1.096199,1,4.096199
Parasailing and Paragliding,1,1,1.063425,1,4.063425
Sporting Days Out,1,1,1.046068,1,4.046068
Sprinter Merceded Van,1,1,1.037471,1,4.037471
Family,1,1,1.018454,1,4.018454
Other Outdoor Activities,1,1,1.011932,1,4.011932
Family Care,1,1,1.000000,1,4.000000


In [53]:
title = 'body massage'
description = 'foot massage in singapore central mall'
image_path = None
video_path = 'https://media.istockphoto.com/videos/full-body-massage-video-id158999993'
# https://www.youtube.com/watch?v=Q8JtnSD2Ync

get_recommendations()

Analysing 10.00s-11.60s clip segment | Segment 1/3
Analysing 11.60s-13.20s clip segment | Segment 2/3
Analysing 13.20s-14.80s clip segment | Segment 3/3


,title,description,image,massaging legs,massaging back,massaging feet,waxing legs,applying cream,final_score
Foot Massage,1.311375,10.000000,1,10.000000,1.153787,10.000000,1.015660,1.000000,81.819715
Back Massage,10.000000,1.000000,1,1.093435,10.000000,1.026848,1.000000,1.000000,54.360849
Cooking and Whipping Cream,1.000000,1.000000,1,1.000000,1.000000,1.000000,1.000000,10.000000,45.000000
Waxing and Hair Removal,1.000000,1.000000,1,1.000000,1.000000,1.000000,10.000000,1.000000,45.000000
Hand Massage,3.632692,1.090091,1,1.431489,1.230738,1.207078,1.000000,1.000000,23.330698
Women's Shaving and Hair Removal,1.000000,1.000000,1,1.000000,1.000000,1.000000,1.984046,1.000000,20.952137
Ice Cream,1.000000,1.000000,1,1.000000,1.000000,1.000000,1.000000,1.629698,19.889094
Pedicure,1.000000,1.000000,1,1.000000,1.000000,1.274030,1.206397,1.000000,19.441281
Facial Massage,1.507366,1.000000,1,1.129878,1.158391,1.002980,1.000000,1.007835,19.404617
Mens Shaving,1.000000,1.000000,1,1.000000,1.000000,1.000000,1.371478,1.000000,19.114434


In [65]:
title = None
description = None
image_path = None
video_path = 'cktlc2t6d0txb01k67jk8a62b.mp4'

get_recommendations()

../../data/artifacts/raw/videos/adventures/cktlc2t6d0txb01k67jk8a62b.mp4
Analysing 10.00s-11.60s clip segment | Segment 1/3
Analysing 11.60s-13.20s clip segment | Segment 2/3
Analysing 13.20s-14.80s clip segment | Segment 3/3


,title,description,image,unloading truck,building cabinet,driving car,motorcycling,digging,using remote controller (not gaming),unboxing,news anchoring,crossing river,final_score
Ram Trucks,1,1,1,10.000000,1.000000,1,1,1,1,1.000000,1,1,45.000000
Vehicle Transmission,1,1,1,1.567288,1.000000,10,1,1,1,1.000000,1,1,37.701865
Shopping,1,1,1,1.000000,1.000000,1,1,10,1,1.037726,1,1,36.037726
Motorcycle Tour,1,1,1,1.000000,1.000000,1,10,1,1,1.000000,1,1,36.000000
Team Building,1,1,1,1.000000,10.000000,1,1,1,1,1.000000,1,1,36.000000
River Cruises,1,1,1,1.000000,1.000000,1,1,1,1,1.000000,1,10,27.000000
Room Escape Games,1,1,1,1.000000,1.000000,1,1,1,10,1.000000,1,1,27.000000
AN-12,1,1,1,1.000000,1.000000,1,1,1,1,10.000000,1,1,27.000000
Ties,1,1,1,1.000000,1.000000,1,1,1,1,1.000000,10,1,27.000000
Cupboard Foods,1,1,1,1.000000,3.521455,1,1,1,1,1.000000,1,1,23.042910
